In [3]:
import urllib.request as ul
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import datetime
import pandas as pd
import json
import csv

#Stripper dependencies
import sys 
sys.modules["regex"] = __import__("re")
import nltk
nltk.data.path.append(".")
try:
    zyx = nltk.data.find('averaged_perceptron_tagger')
    wrt = nltk.data.find('tokenizers/punkt')
except:
    nltk.download("punkt", download_dir=".")
    nltk.download("averaged_perceptron_tagger", download_dir=".")
from collections import Counter
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
#---

senatorwiki = 'https://en.wikipedia.org/wiki/List_of_current_United_States_Senators'
html = requests.get(senatorwiki)
soup = BeautifulSoup(html.text)
senatortable = soup.find('table',{'class':"sortable"})
rows = senatortable.findAll('tr')

state = []
name = []
party = []
age = []
occupation = []
prev = []
assoff = []
termup = []
res = []

for tr in rows:
    cells=tr.findAll('td')
    alt = tr.findAll('th')
    if len(cells)==9:
        #state.append(cells[0].find(text=True))
        party.append(cells[2].find(text=True))
        occupation.append(cells[4].find(text=True))
        prev.append(cells[5].find(text=True))
        assoff.append(cells[6].find(text=True))
        termup.append(cells[7].find(text=True))
        res.append(cells[8].find(text=True))
        age.append(int(cells[3].find('span','noprint ForceAgeToShow').string[6:-1]))
    if len(cells)==10:
        state.append(cells[0].find(text=True))
        party.append(cells[3].find(text=True))
        occupation.append(cells[5].find(text=True))
        prev.append(cells[6].find(text=True))
        assoff.append(cells[7].find(text=True))
        termup.append(cells[8].find(text=True))
        res.append(cells[9].find(text=True))
        age.append(int(cells[4].find('span','noprint ForceAgeToShow').string[6:-1]))
    if len(alt)==1:
        name.append(alt[0].find(text=True))

c=rows[0].findAll('th')
stac = [el.find(text=True) for el in c]
cats = [el.rstrip() for el in stac]
cats.remove('Portrait')
cats[3] = 'Age'
zcat = dict(zip([str(x) for x in range(len(cats))],cats))

assoff = [x.rstrip() for x in assoff]
assoff = [datetime.datetime.strptime(x,'%B %d, %Y') for x in assoff]
occupation = [x.rstrip() for x in occupation]
termup = [x.rstrip() for x in termup]
termup = [datetime.datetime.strptime(x,'%Y') for x in termup]
states = [x for x in state for i in range(2)]
prev = [x.rstrip() for x in prev]

senators = pd.DataFrame([states,name,party,age,occupation,prev,assoff,termup,res],index=cats).T

today = datetime.date.today()
yrs = pd.Series([senators['Assumed office'][x].year for x in range(len(senators))])

senators['Years served'] = today.year - yrs

#takes the url tails for each senator
url = 'https://justfacts.votesmart.org/candidate/public-statements'
search = 'https://votesmart.org/search?q='
allsen ='https://votesmart.org/officials/NA/C/-congressional?chamber=U'
html = requests.get(allsen)
soup = BeautifulSoup(html.text)
f = soup.find('h5').find('a')['href'][10:]
soup.findAll('h5')
sencodedict = {line.text:line.find('a')['href'][10:] for line in soup.findAll('h5')}

#updates names to match votesmart
seniority = pd.Series([1 if x%2 else 0 for x in range(1,101)])
senators['Seniority'] = seniority
names = senators['Senator']
surnames = [name.split(' ')[-1] if name.split(' ')[-1] != 'Jr.' else name.split(' ')[-2] for name in names]
senators['Surname'] = surnames
senators = senators.sort_values(by='Surname')

# Alphabetized from votesmart.
sendict = list(sencodedict.keys())
rep = list(sencodedict.keys())
for x in range(len(rep)):
    rep[x] = rep[x].split(' ')
rep = sorted(rep,key=lambda x: x[-1] if (x[-1] != 'Jr.') and (x[-1] != 'III') else x[-2])
for x in range(len(rep)):
    rep[x] = ' '.join(rep[x])
    
senators['Senator'] = rep

alphalist = list(senators['Senator'])
femwiki = 'https://en.wikipedia.org/wiki/Women_in_the_United_States_Senate'
femhtml = requests.get(femwiki)
chickensoup = BeautifulSoup(femhtml.text)
femtable = chickensoup.find('table',{'class':"sortable"})
frows = femtable.findAll('tr')
ttxt = []
for line in femtable.findAll('a'):
    ttxt.append(line.text)
femlist = [x for x in alphalist if x.split(' ')[-1] in [x.split(' ')[-1] for x in ttxt]]
gender = [1 if x not in femlist else 0 for x in alphalist]
senators['Gender'] = gender

party = {}
party[senators['Party'].unique()[0]] = 1
party[senators['Party'].unique()[1]] = -1
party[senators['Party'].unique()[2]] = 0
party_list = [party[x] for x in senators['Party']]
senators['Party num'] = party_list


In [4]:
senators.to_csv('wiki.csv')

In [49]:
# Crawls and snaps floor speeches
sendict = list(sencodedict.keys())
speeches = {}
for x in sendict:
    speeches[x] = []
for x in range(len(sendict)):
    track = requests.get(url+sencodedict[sendict[x]]+'?speechType=14')
    borsch = BeautifulSoup(track.text)
    for link in borsch.findAll('td',{'class':'statements-table-data'}):
        te = link.find('a')['href']
        kharcho = BeautifulSoup(requests.get(te).text)
        try:
            speeches[sendict[x]].append(kharcho.find('div',{'id':"publicStatementDetailSpeechContent"}).text)
        except:
            speeches[sendict[x]].append('')
    print('{}'.format((x+1)/100))

0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
1.0


In [32]:
###Troubleshooting
track = requests.get(url+sencodedict[sendict[2]]+'?speechType=14')
borsch = BeautifulSoup(track.text)
test=[]
for link in borsch.findAll('td',{'class':'statements-table-data'}):
    te = link.find('a')['href']
    kharcho = BeautifulSoup(requests.get(te).text)
    try:
        speeches[sendict[x]].append(kharcho.find('div',{'id':"publicStatementDetailSpeechContent"}).text)
    except:
        speeches[sendict[x]].append('')
test[0].find('div',{'id':"publicStatementDetailSpeechContent"}).text

In [55]:
job = speeches

In [56]:
bit = 'BREAK IN TRANSCRIPT'

#first clipping -- ONLY RUN ONCE
for y in range(len(sendict)):
    for x in range(len(job[sendict[y]])):
        test = job[sendict[y]][x]
        job[sendict[y]][x] = test[test.find(bit)+len(bit):test.rfind(bit)]

# fully cleaned texts
cleaned = {}
for y in range(len(sendict)):
    for x in range(len((job[sendict[y]]))):
        job[sendict[y]][x] = job[sendict[y]][x].split(bit)
    testlist = [item for sublist in job[sendict[y]] for item in sublist]
    cleaned[sendict[y]] = [item for item in testlist if item != '']
    
# jobject = json.dumps(cleaned)
# with open('clean_speeches_2021.json','x') as file:
#     file.write(jobject)

In [58]:
jobject = json.dumps(cleaned)
with open('clean_speeches_2021.json','x') as file:
    file.write(jobject)

In [60]:
# makes dictionary from csv/txt
def webster(file, type=1):
    if type == 1:
        dic = dict(csv.reader(open(file, 'r')))
    else:
        dic = dict([line.split() for line in open(file).read().replace('\t', ' ').split('\n') if len(line) != 0])
    for key in dic:
        dic[key] = float(dic[key])
    return dic

# subsection - dictionaries
valence = webster('valence.csv', type=1)
arousal = webster('arousal.csv', type=1)
dominance = webster('dominance.csv', type=1)
anger = webster('anger-scores.txt', type=0)
sadness = webster('sadness-scores.txt', type=0)
anticipation = webster('anticipation-scores.txt', type=0)
disgust = webster('disgust-scores.txt', type=0)
fear = webster('fear-scores.txt', type=0)
joy = webster('joy-scores.txt', type=0)
surprise = webster('surprise-scores.txt', type=0)
trust = webster('trust-scores.txt', type=0)
humor = pd.read_csv('humor_dataset.csv')
humor_tot = dict(zip(humor['word'], humor['mean']))
humor_M = dict(zip(humor['word'], humor['mean_M']))
humor_F = dict(zip(humor['word'], humor['mean_F']))
humor_young = dict(zip(humor['word'], humor['mean_young']))
humor_old = dict(zip(humor['word'], humor['mean_old']))
selfish = ['I’m', "i'll", 'i’ll', 'me', 'I’ll', 'i’d', "i'd", 'my', 'myself', 'i’m', "i'm", 'i’ve', "i've", 'I', 'i’d’ve', "i'd've", 'i', 'mine']

# create local / global averages
def emoav(emo, selfemo, wrdct):
    emo_len = len(selfemo)
    perc_emo = emo_len / wrdct
    scores = [emo[word] for word in selfemo]
    if len(scores) == 0:
      return 0, 0
    else:
      avg_loc = np.mean(scores)
      avg_glo = avg_loc * perc_emo
      return avg_loc, avg_glo

class Stripper:
    """For getting basic info from a file"""
    def __init__(self, text, person, age, gender):
        # imported info from excel
        self.stripped = text
        self.person = person
        self.age = age
        self.gender = gender       

        # top-line features
        sss = nltk.tokenize.sent_tokenize(self.stripped)
        self.sents = [x for x in sss if x[0].isalpha()]
        self.pos_sents = [nltk.pos_tag(self.sents[i].split(' ')) for i in range(len(self.sents)) if '-----------------' not in self.sents[i]]
        self.num_sents = len(self.sents)
        self.words_tok = nltk.tokenize.word_tokenize(self.stripped)
        self.words_raw = [x for x in self.stripped.split(' ')]
        self.wordcount = len(self.words_raw)
        self.clean_words = [word.lower().replace('.', '').replace(',', '') for word in self.words_raw]
        self.diff_words = set(self.clean_words)
        
        # POS features
        self.pos = nltk.pos_tag(self.words_tok)
        self.counts = Counter(tag for word,tag in self.pos)
        r = 0
        for key in self.counts:
            r += self.counts[key]
        r = r - (self.counts[','] + self.counts['.'])
        # all individual counts are relative to *pos counts* (not wordcounts)
        self.noun = 100 * (self.counts['NN'] + self.counts['NNS'] + self.counts['NNP'] + self.counts['NNPS']) / r
        self.pron = 100 * (self.counts['PRP'] + self.counts['PRP$']) / r
        self.adj = 100 * (self.counts['JJ'] + self.counts['JJR'] + self.counts['JJS']) / r
        self.adv = 100 * (self.counts['RB'] + self.counts['RBR'] + self.counts['RBS']) / r
        self.intj = 100 * (self.counts['UH']) / r
        self.verb = 100 * (self.counts['VB'] + self.counts['VBD'] + self.counts['VBG'] + self.counts['VBN'] + self.counts['VBP'] + self.counts['VBZ']) / r
        self.wh = 100 * (self.counts['WDT'] + self.counts['WP'] + self.counts['WP$'] + self.counts['WRB']) / r
        self.conj = 100 * (self.counts['CC']) / r
        self.prep = 100 * (self.counts['IN'] + self.counts['TO']) / r
        
        # fragments
        self.frag = 0
        self.sent_counts = [Counter(tag for word, tag in sent) for sent in self.pos_sents]
        verbs = {'VB','VBD','VBG','VBN','VBP','VBZ'}
        for sent in self.sent_counts:
            if len(verbs-set(sent.keys())) == len(verbs):
                self.frag += 1
                
        # word frequencies
        self.freq = nltk.FreqDist(word for word in self.clean_words)
        func = open('funcwords.txt').read().translate(str.maketrans("',[]", '    ')).split()
        self.funcs_freq = 100 * sum(self.freq[word] for word in self.freq if word in func) / self.wordcount
        self.rare = [word for word in self.freq if self.freq[word] == 1]
        self.rare_nonfunc = [word for word in self.rare if word not in func]
        self.selfish = sum(self.freq[word] for word in self.freq if word in selfish) / self.num_sents
        
        self.anger = [word for word in self.clean_words if word in anger.keys()]
        self.sadness = [word for word in self.clean_words if word in sadness.keys()]
        self.anticipation = [word for word in self.clean_words if word in anticipation.keys()]
        self.disgust = [word for word in self.clean_words if word in disgust.keys()]
        self.fear = [word for word in self.clean_words if word in fear.keys()]
        self.joy = [word for word in self.clean_words if word in joy.keys()]
        self.surprise = [word for word in self.clean_words if word in surprise.keys()]
        self.trust = [word for word in self.clean_words if word in trust.keys()]
        
        self.valence = [word for word in self.clean_words if word in valence.keys()]
        self.arousal = [word for word in self.clean_words if word in arousal.keys()]
        self.dominance = [word for word in self.clean_words if word in dominance.keys()]

        self.humor = [word for word in self.clean_words if word in humor_tot.keys()]
        
        # emotional strength
        self.al_fear, self.ag_fear = emoav(fear, self.fear, self.wordcount)
        self.al_joy, self.ag_joy = emoav(joy, self.joy, self.wordcount)
        self.al_trust, self.ag_trust = emoav(trust, self.trust, self.wordcount)
        self.al_surprise, self.ag_surprise = emoav(surprise, self.surprise, self.wordcount)
        self.al_disgust, self.ag_disgust = emoav(disgust, self.disgust, self.wordcount)
        self.al_anticipation, self.ag_anticipation = emoav(anticipation, self.anticipation, self.wordcount)
        self.al_anger, self.ag_anger = emoav(anger, self.anger, self.wordcount)
        self.al_sadness, self.ag_sadness = emoav(sadness, self.sadness, self.wordcount)

        self.al_valence, self.ag_valence = emoav(valence, self.valence, self.wordcount)
        self.al_arousal, self.ag_arousal = emoav(arousal, self.arousal, self.wordcount)
        self.al_dominance, self.ag_dominance = emoav(dominance, self.dominance, self.wordcount)

        self.al_humor_tot, self.ag_humor_tot = emoav(humor_tot, self.humor, self.wordcount)
        self.al_humor_M, self.ag_humor_M = emoav(humor_M, self.humor, self.wordcount)
        self.al_humor_F, self.ag_humor_F = emoav(humor_F, self.humor, self.wordcount)
        self.al_humor_young, self.ag_humor_young = emoav(humor_young, self.humor, self.wordcount)
        self.al_humor_old, self.ag_humor_old = emoav(humor_old, self.humor, self.wordcount)

        # sentence and word size
        self.short_sent, self.long_sent = 0, 0
        for sent in self.sents:
            if len(sent.split(' ')) <= 6:
                self.short_sent += 1
            else:
                self.long_sent += 1

        self.short_word, self.long_word = 0, 0
        for word in self.clean_words:
            if len(word) <= 6:
                self.short_word += 1
            else:
                self.long_word += 1
    
    # returns numpy array
    def bare(self):
        return np.array([self.person, self.age, self.gender, self.num_sents, self.wordcount, self.noun, self.verb, self.adj, self.adv, self.intj, self.pron, self.wh, self.conj, self.prep, int(self.short_sent), int(self.long_sent), self.short_word, self.long_word, self.frag, len(self.diff_words), self.funcs_freq, self.selfish, len(self.rare), (100*len(self.rare_nonfunc)/self.wordcount), (100*len(self.joy)/self.wordcount), (100*len(self.surprise)/self.wordcount), (100*len(self.anticipation)/self.wordcount), (100*len(self.anger)/self.wordcount), (100*len(self.fear)/self.wordcount), (100*len(self.trust)/self.wordcount), (100*len(self.disgust)/self.wordcount), (100*len(self.sadness)/self.wordcount), self.al_fear, self.ag_fear, self.al_joy, self.ag_joy, self.al_trust, self.ag_trust, self.al_surprise, self.ag_surprise, self.al_disgust, self.ag_disgust, self.al_anticipation, self.ag_anticipation, self.al_anger, self.ag_anger, self.al_sadness, self.ag_sadness, self.al_valence, self.ag_valence, self.al_arousal, self.ag_arousal, self.al_dominance, self.ag_dominance, self.al_humor_tot, self.ag_humor_tot, self.al_humor_M, self.ag_humor_M, self.al_humor_F, self.ag_humor_F, self.al_humor_young, self.ag_humor_young, self.al_humor_old, self.ag_humor_old, (100*len(self.humor)/self.wordcount), (100*len(self.valence)/self.wordcount)])
        
# converts arrays into dataframe        
def cashout(arrays):
    temp = [[arrays[i].bare()] for i in range(len(arrays))]
    s = pd.DataFrame(np.concatenate(temp), columns=['person', 'age', 'gender', 'sents','wordcount','noun','verb','adj','adv','intj','pron','wh','conj','prep','short sent','long sent','short word','long word','frag','# diff','% func words', 'selfish', '# unique', '% unique no func', '% joy', '% surprise', '% anticipation', '% anger', '% fear', '% trust', '% disgust', '% sadness', 'loc fear', 'glb fear', 'loc joy', 'glb joy', 'loc trust', 'glb trust', 'loc surprise', 'glb surprise', 'loc disgust', 'glb disgust', 'loc anticipation', 'glb anticipation', 'loc anger', 'glb anger', 'loc sadness', 'glb sadness', 'loc valence', 'glb valence', 'loc arousal', 'glb arousal', 'loc dominance', 'glb dominance', 'loc humor', 'glb humor', 'loc humor M', 'glb humor M', 'loc humor F', 'glb humor F', 'loc humor young', 'glb humor young', 'loc humor old', 'glb humor old', '% humor', '% VAD'])

    # type conversions
    stringcol = ['person']
    intcol = ['age', 'gender', 'sents', 'wordcount', 'short sent', 'long sent','short word', 'long word', 'frag', '# diff']
    floatcol = ['noun', 'verb', 'adj', 'adv', 'intj', 'pron', 'wh', 'conj', 'prep','% func words', 'selfish','# unique', '% unique no func', '% joy', '% surprise', '% anticipation','% anger', '% fear', '% trust', '% disgust', '% sadness', 'loc fear','glb fear', 'loc joy', 'glb joy', 'loc trust', 'glb trust','loc surprise', 'glb surprise', 'loc disgust', 'glb disgust','loc anticipation', 'glb anticipation', 'loc anger', 'glb anger','loc sadness', 'glb sadness', 'loc valence', 'glb valence','loc arousal', 'glb arousal', 'loc dominance', 'glb dominance','loc humor', 'glb humor', 'loc humor M', 'glb humor M', 'loc humor F','glb humor F', 'loc humor young', 'glb humor young', 'loc humor old','glb humor old', '% humor', '% VAD'] 
    s[stringcol] = s[stringcol].astype(str)
    s[intcol] = s[intcol].astype(str).astype(int)
    s[floatcol] = s[floatcol].astype(str).astype(float)
    return s

# calculates composite measures into the dataframe
def dance(ndf):
    ndf['short/long'] = ndf['short sent'] / ndf['long sent']
    ndf['words/sent'] = ndf['wordcount'] / ndf['sents']
    ndf['conj/sent'] = ndf['conj'] / ndf['sents']
    ndf['verb/noun'] = ndf['verb'] / ndf['noun']
    ndf['noun/pron'] = ndf['noun'] / ndf['pron']
    ndf['% diff w/func'] = ndf['# diff'] / ndf['wordcount'] * 100
    ndf['% unique w/func'] = ndf['# unique'] / ndf['wordcount'] * 100
    ndf['misc words'] = ndf['wordcount'] - (ndf['noun'] + ndf['verb'] + ndf['adj'] + ndf['adv'] + ndf['pron'] + ndf['intj'] + ndf['wh'] + ndf['prep'] + ndf['conj'])
    ndf['% frag'] = ndf['frag'] / ndf['sents'] * 100
    
# does all of the above in one line with the option of switching to manipulate a single object
def VIP(dic):
    temp = cashout(dic)
    dance(temp)
    return temp
    
def tablemaker(atts,name,alsoticks='False',time='False'):
    if time == 'False':
        bx = biden[atts].mean()
        tx = trump[atts].mean()
        bxerr = biden[atts].std()/sqrt(len(biden.index))
        txerr = trump[atts].std()/sqrt(len(trump.index))
    else:
        bx = biden_timed[atts].mean()
        tx = trump_timed[atts].mean()
        bxerr = biden_timed[atts].std()/sqrt(len(biden_timed.index))
        txerr = trump_timed[atts].std()/sqrt(len(trump_timed.index))
        

    Z = np.arange(len(atts))
    plt.bar(Z-.1, bx, color='#0015BC', width = .2, label='Biden', yerr=bxerr)
    plt.bar(Z+.1, tx, color='#DE0100', width = .2, label='Trump', yerr=txerr)

    plt.ylabel('Score')
    plt.title(name)
    if alsoticks=='False':
        plt.xticks(Z, tuple(atts))
    else:
        plt.xticks(Z, tuple(alsoticks))
    plt.legend(loc='best')
    plt.figure(figsize=(30,20))
#     plt.savefig(name+'.png')
    plt.show()

In [61]:
with open('clean_speeches_2021.json','r') as openfile:
    cleanjob = json.load(openfile)

for y in range(len(sendict)):
    for x in range(len(cleanjob[sendict[y]])):
        test = cleanjob[sendict[y]][x]
        cleanjob[sendict[y]][x] = test.replace('\n',' ').replace('\n ',' ').replace('  \n',' ').replace('   ',' ').replace('    ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').strip()
    cleanjob[sendict[y]] = [u for u in cleanjob[sendict[y]] if len(u.split(' ')) > 20]
    cleanjob[sendict[y]] = [u for u in cleanjob[sendict[y]] if 'M' in u[:2]]

In [64]:
allsens = []
for y in range(len(sendict)):
    print('y-{}--sen-{}'.format(y,sendict[y]))
    for x in range(len(cleanjob[sendict[y]])):
        allsens.append(Stripper(cleanjob[sendict[y]][x], sendict[y], int(senators[senators['Senator']==sendict[y]]['Age']), int(senators[senators['Senator']==sendict[y]]['Gender'])))
        print('{} of {}'.format(1+x,len(cleanjob[sendict[y]])-1))
    

y-0--sen-John Barrasso
0 of 20
1 of 20
2 of 20
3 of 20
4 of 20
5 of 20
6 of 20
7 of 20
8 of 20
9 of 20
10 of 20
11 of 20
12 of 20
13 of 20
14 of 20
15 of 20
16 of 20
17 of 20
18 of 20
19 of 20
20 of 20
y-1--sen-Michael Bennet
0 of 29
1 of 29
2 of 29
3 of 29
4 of 29
5 of 29
6 of 29
7 of 29
8 of 29
9 of 29
10 of 29
11 of 29
12 of 29
13 of 29
14 of 29
15 of 29
16 of 29
17 of 29
18 of 29
19 of 29
20 of 29
21 of 29
22 of 29
23 of 29
24 of 29
25 of 29
26 of 29
27 of 29
28 of 29
29 of 29
y-2--sen-Marsha Blackburn
0 of 17
1 of 17
2 of 17
3 of 17
4 of 17
5 of 17
6 of 17
7 of 17
8 of 17
9 of 17
10 of 17
11 of 17
12 of 17
13 of 17
14 of 17
15 of 17
16 of 17
17 of 17
y-3--sen-Richard Blumenthal
0 of 36
1 of 36
2 of 36
3 of 36
4 of 36
5 of 36
6 of 36
7 of 36
8 of 36
9 of 36
10 of 36
11 of 36
12 of 36
13 of 36
14 of 36
15 of 36
16 of 36
17 of 36
18 of 36
19 of 36
20 of 36
21 of 36
22 of 36
23 of 36
24 of 36
25 of 36
26 of 36
27 of 36
28 of 36
29 of 36
30 of 36
31 of 36
32 of 36
33 of 36
34 of 36
35 

In [65]:
ALLSENS = VIP(allsens)
ALLSENS.to_pickle('allsens_2021.pkl')

In [66]:
tot = 0
for x in sendict:
    print('{}: {} texts'.format(x,len(cleanjob[x])))
    tot += len(cleanjob[x])
print(tot, tot/100)

John Barrasso: 21 texts
Michael Bennet: 30 texts
Marsha Blackburn: 18 texts
Richard Blumenthal: 37 texts
Roy Blunt: 20 texts
John Boozman: 8 texts
Sherrod Brown: 33 texts
Richard Burr: 28 texts
Ben Cardin: 31 texts
Tom Carper: 36 texts
Bob Casey, Jr.: 26 texts
Bill Cassidy: 16 texts
Susan Collins: 23 texts
John Cornyn: 19 texts
Catherine Cortez Masto: 0 texts
Mike Crapo: 29 texts
Dick Durbin: 20 texts
Dianne Feinstein: 26 texts
Deb Fischer: 18 texts
Lindsey Graham: 34 texts
Chuck Grassley: 24 texts
John Hoeven: 45 texts
Jim Inhofe: 20 texts
Ron Johnson: 38 texts
Amy Klobuchar: 36 texts
Patrick Leahy: 23 texts
Mike Lee: 45 texts
Ben Lujan, Jr.: 39 texts
Joe Manchin III: 32 texts
Mitch McConnell, Jr.: 18 texts
Bob Menendez: 36 texts
Jerry Moran: 12 texts
Lisa Murkowski: 21 texts
Patty Murray: 28 texts
Jon Ossoff: 0 texts
Jack Reed: 23 texts
Marco Rubio: 24 texts
Brian Schatz: 26 texts
Chuck Schumer: 18 texts
Jeanne Shaheen: 30 texts
Richard Shelby: 25 texts
Kyrsten Sinema: 39 texts
Debbi

In [9]:
tot = 0
for x in sendict:
    print('{}: {} texts'.format(x,len(cleanjob[x])))
    tot += len(cleanjob[x])
print(tot, tot/100)

Lamar Alexander: 33 texts
Michael Bennet: 32 texts
Richard Blumenthal: 27 texts
Roy Blunt: 24 texts
John Boozman: 8 texts
Sherrod Brown: 32 texts
Richard Burr: 27 texts
Ben Cardin: 30 texts
Tom Carper: 36 texts
Bob Casey, Jr.: 28 texts
Bill Cassidy: 19 texts
Susan Collins: 24 texts
John Cornyn: 28 texts
Catherine Cortez Masto: 0 texts
Mike Crapo: 31 texts
Dick Durbin: 23 texts
Mike Enzi: 24 texts
Dianne Feinstein: 28 texts
Deb Fischer: 20 texts
Lindsey Graham: 34 texts
Chuck Grassley: 25 texts
John Hoeven: 35 texts
Jim Inhofe: 28 texts
Ron Johnson: 43 texts
Amy Klobuchar: 37 texts
Patrick Leahy: 30 texts
Mike Lee: 47 texts
Joe Manchin III: 41 texts
Mitch McConnell, Jr.: 7 texts
Bob Menendez: 37 texts
Lisa Murkowski: 26 texts
Patty Murray: 26 texts
David Perdue: 11 texts
Jack Reed: 27 texts
Pat Roberts: 40 texts
Marco Rubio: 27 texts
Brian Schatz: 26 texts
Chuck Schumer: 23 texts
Jeanne Shaheen: 29 texts
Richard Shelby: 29 texts
Kyrsten Sinema: 39 texts
Debbie Stabenow: 33 texts
Jon Tes